In [1]:
import ASW
import mat_properties as prop
import pandas as pd
import SP

RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
water = prop.Materials_prop(
    "WATER",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
accumulation = pd.read_excel("blocks.xlsx", sheet_name="accumulation", index_col=0)
ASWatm = True
ASWbul = 1
vremya = 4
vremyaojidaniya=12
water_streams.at["AIR", "T"] = -29
htest1 = water.p_t(1, 95)['h']
print(htest1)
htest1 = water.p_t(0.1, 95)['h']
print(htest1)
# P = 2800

 # таблица номинального режима
gas_streams0 = pd.read_excel(
    "streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel(
    "streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel(
    "input.xlsx", sheet_name="GTU_input", index_col=0)
GTU_input.at["n", 1] = 1
GTU_input.at["tair", 1] = water_streams.at["AIR", "T"]

# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel(
    "streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)
accumulation = pd.read_excel(
    "blocks.xlsx", sheet_name="accumulation", index_col=0)
# Теплосеть и перекидка температуры воздуха
gas_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
water_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
Tnv = water_streams.at["AIR", "T"]
water_streams.at["SWIN", "T"] = SP.Tset(Tnv)[1]
water_streams.at["SWOUT", "T"] = SP.Tset(Tnv)[0]


#for check
water_streams.at["SWIN-TURB", "H"] = water.p_t(1, SP.Tset(Tnv)[1])['h']
water_streams.at["SP2-WOUT", "H"] = water.p_t(1, SP.Tset(Tnv)[0])['h']


water_streams.at["SWIN-TURB", "T"] = water_streams.at["SWIN", "T"]
water_streams.at["SWIN-TURB", "G"] = water_streams.at["SWIN", "G"]
water_streams.at["SP2-WOUT", "T"] = water_streams.at["SWOUT", "T"]

398.787163068191
398.1007015109759


In [2]:
import numpy as np
from numpy import linalg as LA
# ----------------------------- Начало Максима Опарина
ASW = ASW.Accum(water,water_streams,accumulation,ASWatm,
                     stream12 = "ASW-WOUT",
                     stream11 = "SP2-ASW",
                     stream_obratnoi_setevoi_vody = "SWIN-TURB",
                     stream_pryamoi_setevoi_vody = "SP2-WOUT",
#                      T_nar_vozd = water_streams.at["AIR", "T"]
                    T_nar_vozd = -25
               )

ASW.set_construct(Diametr=6,
                       kolichestvo = 2,
                       Visota = 10,
                       lambda_min_vata = 0.045,
                       delta_min_vata = 0.01)

if ASWatm == False:
    
    
    # # если Зарядка   
    if ASWbul == 1:
        G_ASW_zarydka = ASW.zaryadka(vremya)['G']
        print(G_ASW_zarydka,'G')
        water_streams.at["SWIN-TURB", "G"] = water_streams.at["SWIN", "G"] + G_ASW_zarydka
        water_streams.at["SP2-WOUT", "G"] = water_streams.at["SWIN", "G"] + G_ASW_zarydka
            ## если разрядка
    if ASWbul == 2:
        ASW.zaryadka(vremya)
        ASW.jdat_n(vremyaojidaniya)
        G_ASW_razryadka = ASW.razryadka(vremya)['G']
        print(G_ASW_razryadka,'check')
        water_streams.at["SWIN-TURB", "G"] = water_streams.at["SWIN", "G"] - G_ASW_razryadka
        water_streams.at["SP2-WOUT", "G"] = water_streams.at["SWIN", "G"] - G_ASW_razryadka
        water_streams.at["SWOUT", "H"] = water.p_t(water_streams.at["SWOUT","P"],water_streams.at["SWOUT", "T"])['h']
        print(water_streams.at["SWOUT", "P"], 'Check P')
        print(water_streams.at["SWOUT", "T"], 'Check T')  
        print(water_streams.at["SWOUT", "H"],'check H')
        water_streams.at["SP2-WOUT", "H"] = (water_streams.at["SWOUT", "H"]*water_streams.at["SWOUT", "G"] -
                                            water_streams.at["ASW-WOUT", "H"]*G_ASW_razryadka)/(water_streams.at["SWOUT", "G"]-G_ASW_razryadka)
        water_streams.at["SP2-WOUT", "T"] = water.p_h(water_streams.at["SP2-WOUT", "P"],water_streams.at["SP2-WOUT", "H"])['T']
else:

    # # если Зарядка   
    if ASWbul == 1:
        G_ASW_zarydka = ASW.zaryadka(vremya)['G']
        print(G_ASW_zarydka,'GATM')
        
        a = [[1, 1],
        [1*water_streams.at["SP2-WOUT", "H"], 1*water_streams.at["SWIN-TURB", "H"]]]
#         b = [G_ASW_zarydka, G_ASW_zarydka*water.p_t(0.1,95)['h']]
        b = [G_ASW_zarydka, G_ASW_zarydka*water.p_t(0.1,min(95,water_streams.at["SP2-WOUT", "T"]))['h']]
        print(water_streams.at["SP2-WOUT", "H"],water_streams.at["SWIN-TURB", "H"],"Check h posle SP and h pered SP")
        print(G_ASW_zarydka,water.p_t(0.1,95)['h'],"Check G accum and h accum")
        x = LA.solve(a, b)
        G1 = x[0]
        G2 = x[1]
        print(x)
        water_streams.at["SWIN-ASWatm", "G"] = G2 = x[1]
        water_streams.at["SWIN-TURB", "G"] = water_streams.at["SWIN", "G"] + G_ASW_zarydka - G2 
        water_streams.at["SP2-WOUT", "G"] = water_streams.at["SWIN", "G"] + G_ASW_zarydka - G2
            ## если разрядка
    if ASWbul == 2:
        ASW.zaryadka(vremya)
        ASW.jdat_n(vremyaojidaniya)
        G_ASW_razryadka = ASW.razryadka(vremya)['G']
        print(G_ASW_razryadka,'check')
        water_streams.at["SWIN-TURB", "G"] = water_streams.at["SWIN", "G"] - G_ASW_razryadka
        water_streams.at["SP2-WOUT", "G"] = water_streams.at["SWIN", "G"] - G_ASW_razryadka
        water_streams.at["SWOUT", "H"] = water.p_t(water_streams.at["SWOUT","P"],water_streams.at["SWOUT", "T"])['h']
        water_streams.at["SP2-WOUT", "H"] = (water_streams.at["SWOUT", "H"]*water_streams.at["SWOUT", "G"] -
                                            water_streams.at["ASW-WOUT", "H"]*G_ASW_razryadka)/(water_streams.at["SWOUT", "G"]-G_ASW_razryadka)
        water_streams.at["SP2-WOUT", "T"] = water.p_h(water_streams.at["SP2-WOUT", "P"],water_streams.at["SP2-WOUT", "H"])['T']
    
# ----------------------------- Конец Максима Опарина
print(accumulation)
print(water_streams)

120.0 Check T_pr_set_v
T_accum pereopredililas
398.1007015109759 h_accum
0.1 self._P_accum
95 self._T_accum
37.7732260368681 GATM
504.3750257054493 293.85744463923066 Check h posle SP and h pered SP
37.7732260368681 398.1007015109759 Check G accum and h accum
[18.70439554 19.0688305 ]
               Qw     T   V  Qnr
ASW  56701.499107  95.0 NaN  NaN
PKM           NaN   NaN NaN  NaN
                        T         P            H           G   S   X
AIR            -29.000000  0.100000          NaN         NaN NaN NaN
PEVD-DROSVD    511.500000  8.407000  3423.209978   63.400000 NaN NaN
IVD-PEVD       301.297556  8.746000  2747.104023   63.400000 NaN NaN
EVD-IVD        298.800000  8.746000  1337.948265   63.400000 NaN NaN
PEN-EVD        166.500000  8.746000   708.319849   63.400000 NaN NaN
BND-PEN        164.738361  0.696400   696.095567   63.400000 NaN NaN
PPND-DROSND    211.600000  0.691000  2871.481837   14.760000 NaN NaN
IND-PPND       164.738361  0.696400  2762.534980   14.760000 Na

In [3]:
# import mat_properties as prop
# gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# import numpy as np
# from numpy import linalg as LA

# RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
# water = prop.Materials_prop(
#     "water",
#     [1.0, 0, 0, 0, 0],
#     prop.REFPROP_h_s,
#     prop.REFPROP_p_t,
#     prop.REFPROP_p_h,
#     prop.REFPROP_p_s,
#     prop.REFPROP_p_q,
#     prop.REFPROP_t_q,
#     prop.REFPROP_p_rho,
#     prop.REFPROP_s_q,
#     RP=RP,
# )
# Tpr_sv = 120
# Tobr_sv=45
# T_ASW=95
# P_ASW=0.1
# P_pr_sv = 1

# G_ASW= 10

# hpr_sv = water.p_t(P_pr_sv, Tpr_sv)['h']
# print(hpr_sv,'hpr_sv')
# hobr_sv = water.p_t(P_pr_sv, Tobr_sv)['h']
# print(hobr_sv,'hobr_sv')
# h_ASW = water.p_t(P_ASW, T_ASW)['h']
# print(h_ASW,'h_ASW')

# a = [[1, 1],
#      [1*hpr_sv, 1*hobr_sv]]

# b = [G_ASW, G_ASW*h_ASW]

# x = LA.solve(a, b)
# G1 = x[0]
# G2 = x[1]
# print(G1,'vot oni otdelno',G2)

In [4]:
# ASW = ASW.Accum(water,water_streams,accumulation,ASWatm,
#                  stream12 = "ASW-WOUT",
#                  stream11 = "SP2-ASW", # при разряде
#                  stream_obratnoi_setevoi_vody = "SWIN-TURB",
#                  stream_pryamoi_setevoi_vody = "SP2-WOUT",
#                  T_nar_vozd = 15)

# ASW.set_construct(Diametr=2,
#                    kolichestvo = 2,
#                    Visota = 1,
#                    lambda_min_vata = 0.045,
#                    delta_min_vata = 0.01)

# ASW.zaryadka(1)
# ASW.jdat(4)
# ASW.razryadka(1)
# print(water_streams)
# print(heaters)